In [3]:
import pandas as pd

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics
import pickle

In [3]:
print("Tensorlfow version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Tensorlfow version:  2.12.0
Eager mode:  True
GPU is available


In [4]:
# reading the training CSV file
data = pd.read_csv('vulnerability_train.csv')
# displaying the contents of the CSV file


In [5]:
data.head()

,source_code,CWE114,CWE134,CWE190,CWE196,CWE319,CWE369,CWE398,CWE416,CWE427,CWE469,CWE506,CWE605,CWE606,CWE666,CWE680,CWE761,CWE789
0,"char * data ;\nchar dataBuffer [ 250 ] = ""PATH...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,void CWE369_Divide_by_Zero__int_listen_socket_...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,void CWE369_Divide_by_Zero__int_listen_socket_...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,void CWE369_Divide_by_Zero__int_listen_socket_...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,void CWE319_Cleartext_Tx_Sensitive_Info__w32_c...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
columns = data.columns
print(columns)
#data= data.replace({'CWE-119':{'FALSE': False, 'TRUE':True}})

Index(['source_code', 'CWE114', 'CWE134', 'CWE190', 'CWE196', 'CWE319',
       'CWE369', 'CWE398', 'CWE416', 'CWE427', 'CWE469', 'CWE506', 'CWE605',
       'CWE606', 'CWE666', 'CWE680', 'CWE761', 'CWE789'],
      dtype='object')


data_cropped = data.loc[:,['CWE-119', ' CWE-120', 'CWE-469', 'CWE-476', 'CWE-other','functionSource']]

#data_cropped.head(10)
columns = list(data_cropped.columns)
data_cropped = data_cropped.replace({True:"TRUE",False:"FALSE"})
for column in columns[:-1]:
    data_cropped[column] = data_cropped[column].map(str.upper)

In [7]:
data=data.sample(frac=1)

In [28]:
data.head()

,source_code,CWE114,CWE134,CWE190,CWE196,CWE319,CWE369,CWE398,CWE416,CWE427,CWE469,CWE506,CWE605,CWE606,CWE666,CWE680,CWE761,CWE789
17210,static int CVE_2014_2099_PATCHED_msrle_decode_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1055,static TransactionId RecordTransactionCommit()...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10980,void CWE789_Uncontrolled_Mem_Alloc__malloc_cha...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9146,static void goodB2G()\nchar * data ;\nchar dat...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3063,unsigned int avio_rb32(AVIOContext *s)\nunsign...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
data=data.dropna()

In [9]:
myrand=71926
np.random.seed(myrand)
tf.random.set_seed(myrand)
print("Random seed is:",myrand)

Random seed is: 71926


In [10]:
### Set the global value
WORDS_SIZE=10000
INPUT_SIZE=500
NUM_CLASSES=2
MODEL_NUM=0
EPOCHS=10

In [14]:
data['source_code'] = data['source_code'].astype(str)

In [19]:
X=data['source_code']
y=np.array([data['CWE114'].values,data['CWE134'].values,data['CWE190'].values,data['CWE196'].values,data['CWE319'].values,data['CWE369'].values,data['CWE398'].values,data['CWE416'].values,data['CWE427'].values,data['CWE469'].values,data['CWE506'].values,data['CWE605'].values,data['CWE606'].values,data['CWE666'].values,data['CWE680'].values,data['CWE761'].values,data['CWE789'].values])

In [22]:
X.shape

(17724,)

In [25]:
y.shape

(17724, 17)

In [24]:
y = y.T

In [26]:
data.to_pickle("vulnerability_train.pickle")

In [27]:
data=pd.read_pickle("vulnerability_train.pickle")

In [29]:
x_all = data['source_code']

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (17724,)
Shape of y: (17724, 17)


In [35]:
data.columns[1:]

Index(['CWE114', 'CWE134', 'CWE190', 'CWE196', 'CWE319', 'CWE369', 'CWE398',
       'CWE416', 'CWE427', 'CWE469', 'CWE506', 'CWE605', 'CWE606', 'CWE666',
       'CWE680', 'CWE761', 'CWE789'],
      dtype='object')

In [36]:
y_train=[]
#print(data.iloc[:,2])
data = data.ffill()
columns = data.columns[1:]
for column in columns:
    y_train.append(tf.keras.utils.to_categorical(list(data[column].values), num_classes=NUM_CLASSES).astype(np.int64))
y_train = np.array(y_train)

In [37]:
y_train.shape

(17, 17724, 2)

In [38]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False)
tokenizer.fit_on_texts(list(x_all))
tokenizer.num_words = WORDS_SIZE

In [39]:
# data['functionSource'] =  [x.decode('utf-8') for x in data['functionSource'] ]
list_tokenized_train = tokenizer.texts_to_sequences(X)
X = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_train, 
                                  maxlen=INPUT_SIZE,
                                  padding='post')
X = X.astype(np.int64)

In [40]:
X

array([[  31,    8,  338, ...,    0,    0,    0],
       [  31,  465, 2620, ...,    0,    0,    0],
       [  11,  227,  115, ...,    0,    0,    0],
       ...,
       [  11,  227,  115, ...,    0,    0,    0],
       [   5,   85, 2382, ...,    0,    0,    0],
       [  31,   11, 1576, ...,    0,    0,    0]], dtype=int64)

In [41]:
import pickle

#saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
#loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer= pickle.load(handle)

In [42]:
#  Create a random weights matrix
random_weights = np.random.normal(size=(WORDS_SIZE, 13),scale=0.01)

In [43]:
random_weights

array([[ 0.0060252 , -0.00302141, -0.00566813, ..., -0.00071736,
         0.00656931, -0.0077152 ],
       [-0.0026452 ,  0.00101939,  0.00791392, ...,  0.00173955,
         0.00785835,  0.00549122],
       [-0.0044089 ,  0.01024976,  0.01351175, ...,  0.01517417,
        -0.00523813,  0.00256636],
       ...,
       [ 0.0093493 ,  0.0005328 ,  0.00262669, ..., -0.002587  ,
         0.01392652,  0.00944651],
       [-0.00716167,  0.00386448, -0.00194482, ...,  0.00670759,
         0.01145522,  0.00023146],
       [-0.00617482, -0.0155774 ,  0.00127028, ..., -0.00057702,
        -0.00268833, -0.00591792]])

In [44]:
inp_layer = tf.keras.layers.Input(shape=(INPUT_SIZE,))
mid_layers = tf.keras.layers.Embedding(input_dim = WORDS_SIZE,
                                    output_dim = 13,
                                    weights=[random_weights],
                                    input_length = INPUT_SIZE)(inp_layer)
mid_layers = tf.keras.layers.Convolution1D(filters=512, kernel_size=(9), padding='same', activation='relu')(mid_layers)
mid_layers = tf.keras.layers.MaxPool1D(pool_size=5)(mid_layers)
mid_layers = tf.keras.layers.Dropout(0.5)(mid_layers)
mid_layers = tf.keras.layers.Flatten()(mid_layers)
mid_layers = tf.keras.layers.Dense(128, activation='relu')(mid_layers)
mid_layers = tf.keras.layers.Dense(64, activation='relu')(mid_layers)
output1 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output2 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output3 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output4 =tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output5 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output6 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output7 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output8 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output9 =tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output10 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output11 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output12 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output13 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output14 =tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output15 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output16 =tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
output17 = tf.keras.layers.Dense(2, activation='softmax')(mid_layers)
model = tf.keras.Model(inp_layer,[output1,output2,output3,output4,output5,output6,output7,output8,output9,output10,output11,output12,output13,output14,output15,output16,output17])

# Define custom optimizers
# adam = tf.keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1, decay=0.0, amsgrad=False)

## Compile model with metrics
model.compile(optimizer="ADAM", loss='categorical_crossentropy', metrics=['accuracy'])
print("CNN model built: ")
model.summary()

CNN model built: 
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 13)      130000      ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 500, 512)     60416       ['embedding[0][0]']              
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 100, 512)     0           ['conv1d[0][0]']                 
                                                                            

In [45]:
## Create TensorBoard callbacks

callbackdir= 'D:\\temp\\cb'

tbCallback = tf.keras.callbacks.TensorBoard(log_dir=callbackdir, 
                         histogram_freq=1,
                         embeddings_freq=1,
                         write_graph=True, 
                         write_images=True)

tbCallback.set_model(model)
mld = 'model/model-ALL-{epoch:02d}.hdf5'

## Create best model callback
mcp = tf.keras.callbacks.ModelCheckpoint(filepath=mld, 
                                         monitor="val_loss",
                                         save_best_only=True, 
                                         mode='auto', 
                                         save_freq='epoch', 
                                         verbose=1)

In [46]:
y_train.shape

(17, 17724, 2)

In [48]:
y_train

array([[[1, 0],
        [0, 1],
        [1, 0],
        ...,
        [1, 0],
        [0, 1],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       ...,

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [0, 1],
        ...,
        [0, 1],
        [1, 0],
        [1, 0]]], dtype=int64)

In [28]:
class_weights = [{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.}]

callback= tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10, verbose=1, )
history = model.fit(x = X_train,
          y = [y_train[0],y_train[1],y_train[2],y_train[3],y_train[4],y_train[5],y_train[6],y_train[7],y_train[8],y_train[9],y_train[10],y_train[11],y_train[12],y_train[13],y_train[14],y_train[15],y_train[16]],
          epochs = 40,
          batch_size = 128,
          verbose =2,
          callbacks=[callback])

with open('history/History-ALL-40EP-CNN', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

Epoch 1/40
517/517 - 104s - loss: 1.2705 - dense_2_loss: 0.2689 - dense_3_loss: 0.3344 - dense_4_loss: 0.1136 - dense_5_loss: 0.1806 - dense_6_loss: 0.3731 - dense_2_accuracy: 0.8927 - dense_3_accuracy: 0.8612 - dense_4_accuracy: 0.9684 - dense_5_accuracy: 0.9331 - dense_6_accuracy: 0.8363 - 104s/epoch - 202ms/step
Epoch 2/40
517/517 - 105s - loss: 1.1084 - dense_2_loss: 0.2243 - dense_3_loss: 0.2903 - dense_4_loss: 0.1008 - dense_5_loss: 0.1598 - dense_6_loss: 0.3333 - dense_2_accuracy: 0.9126 - dense_3_accuracy: 0.8800 - dense_4_accuracy: 0.9691 - dense_5_accuracy: 0.9406 - dense_6_accuracy: 0.8572 - 105s/epoch - 203ms/step
Epoch 3/40
517/517 - 105s - loss: 0.9799 - dense_2_loss: 0.1912 - dense_3_loss: 0.2545 - dense_4_loss: 0.0903 - dense_5_loss: 0.1420 - dense_6_loss: 0.3020 - dense_2_accuracy: 0.9249 - dense_3_accuracy: 0.8951 - dense_4_accuracy: 0.9704 - dense_5_accuracy: 0.9483 - dense_6_accuracy: 0.8713 - 105s/epoch - 203ms/step
Epoch 4/40
517/517 - 105s - loss: 0.8688 - dense_

Epoch 15/40
517/517 - 102s - loss: 0.3287 - dense_2_loss: 0.0586 - dense_3_loss: 0.0832 - dense_4_loss: 0.0341 - dense_5_loss: 0.0412 - dense_6_loss: 0.1116 - dense_2_accuracy: 0.9783 - dense_3_accuracy: 0.9681 - dense_4_accuracy: 0.9879 - dense_5_accuracy: 0.9844 - dense_6_accuracy: 0.9569 - 102s/epoch - 197ms/step
Epoch 16/40
517/517 - 102s - loss: 0.3077 - dense_2_loss: 0.0540 - dense_3_loss: 0.0766 - dense_4_loss: 0.0326 - dense_5_loss: 0.0391 - dense_6_loss: 0.1055 - dense_2_accuracy: 0.9797 - dense_3_accuracy: 0.9710 - dense_4_accuracy: 0.9886 - dense_5_accuracy: 0.9857 - dense_6_accuracy: 0.9593 - 102s/epoch - 197ms/step
Epoch 17/40
517/517 - 101s - loss: 0.2971 - dense_2_loss: 0.0523 - dense_3_loss: 0.0750 - dense_4_loss: 0.0300 - dense_5_loss: 0.0370 - dense_6_loss: 0.1029 - dense_2_accuracy: 0.9816 - dense_3_accuracy: 0.9713 - dense_4_accuracy: 0.9896 - dense_5_accuracy: 0.9868 - dense_6_accuracy: 0.9607 - 101s/epoch - 196ms/step
Epoch 18/40
517/517 - 101s - loss: 0.2819 - de

Epoch 29/40
517/517 - 101s - loss: 0.1813 - dense_2_loss: 0.0341 - dense_3_loss: 0.0459 - dense_4_loss: 0.0169 - dense_5_loss: 0.0248 - dense_6_loss: 0.0597 - dense_2_accuracy: 0.9878 - dense_3_accuracy: 0.9834 - dense_4_accuracy: 0.9940 - dense_5_accuracy: 0.9916 - dense_6_accuracy: 0.9776 - 101s/epoch - 195ms/step
Epoch 30/40
517/517 - 101s - loss: 0.1804 - dense_2_loss: 0.0343 - dense_3_loss: 0.0466 - dense_4_loss: 0.0165 - dense_5_loss: 0.0237 - dense_6_loss: 0.0593 - dense_2_accuracy: 0.9876 - dense_3_accuracy: 0.9833 - dense_4_accuracy: 0.9940 - dense_5_accuracy: 0.9918 - dense_6_accuracy: 0.9782 - 101s/epoch - 196ms/step
Epoch 31/40
517/517 - 101s - loss: 0.1733 - dense_2_loss: 0.0325 - dense_3_loss: 0.0445 - dense_4_loss: 0.0156 - dense_5_loss: 0.0222 - dense_6_loss: 0.0585 - dense_2_accuracy: 0.9884 - dense_3_accuracy: 0.9837 - dense_4_accuracy: 0.9945 - dense_5_accuracy: 0.9918 - dense_6_accuracy: 0.9789 - 101s/epoch - 195ms/step
Epoch 32/40
517/517 - 102s - loss: 0.1755 - de

FileNotFoundError: [Errno 2] No such file or directory: 'history/History-ALL-40EP-CNN'

In [33]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
data = pd.read_csv('vulnerability_test.csv')


In [52]:
data.head()

,source_code,CWE114,CWE134,CWE190,CWE196,CWE319,CWE369,CWE398,CWE416,CWE427,CWE469,CWE506,CWE605,CWE606,CWE666,CWE680,CWE761,CWE789
3009,"static void mem_add(MemoryListener *listener, ...",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3978,"static\nset_rx_control(E1000State *s, int inde...",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
141,static void CVE_2013_1576_VULN_dissect_sdp_med...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3645,"static void rtl8139_io_writel(void *opaque, ui...",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2115,static inline void xhci_dma_write_u32s(XHCISta...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [51]:
data=data.sample(frac=1)

In [53]:
X=data['source_code']
y=np.array([data['CWE114'].values,data['CWE134'].values,data['CWE190'].values,data['CWE196'].values,data['CWE319'].values,data['CWE369'].values,data['CWE398'].values,data['CWE416'].values,data['CWE427'].values,data['CWE469'].values,data['CWE506'].values,data['CWE605'].values,data['CWE606'].values,data['CWE666'].values,data['CWE680'].values,data['CWE761'].values,data['CWE789'].values])

In [ ]:
y = y.T

In [56]:
y_test=[]
#print(data.iloc[:,2])
data = data.ffill()
columns = data.columns[1:]
for column in columns:
    y_test.append(tf.keras.utils.to_categorical(list(data[column].values), num_classes=NUM_CLASSES).astype(np.int64))
y_test = np.array(y_train)

In [57]:
y_test

array([[[1, 0],
        [0, 1],
        [1, 0],
        ...,
        [1, 0],
        [0, 1],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       ...,

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [0, 1],
        ...,
        [0, 1],
        [1, 0],
        [1, 0]]], dtype=int64)

In [58]:
# data['functionSource'] =  [x.decode('utf-8') for x in data['functionSource'] ]
list_tokenized_test = tokenizer.texts_to_sequences(X)
x_test = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_test, 
                                  maxlen=INPUT_SIZE,
                                  padding='post')
x_test = x_test.astype(np.int64)

In [59]:
x_test

array([[  31,   11,  177, ...,    0,    0,    0],
       [  31,  245,  781, ...,    0,    0,    0],
       [  31,   11,  338, ...,    0,    0,    0],
       ...,
       [  31,  246,  285, ...,    0,    0,    0],
       [  31,   11, 2185, ...,    0,    0,    0],
       [  31,    8,  338, ...,    0,    0,    0]], dtype=int64)

In [60]:
y_test

array([[[1, 0],
        [0, 1],
        [1, 0],
        ...,
        [1, 0],
        [0, 1],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       ...,

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]],

       [[1, 0],
        [1, 0],
        [0, 1],
        ...,
        [0, 1],
        [1, 0],
        [1, 0]]], dtype=int64)

In [ ]:
# Load model
model = tf.keras.models.load_model("svd_1.hdf5")

In [45]:
results = model.evaluate(x_test, y_test = [y_test[0],y_test[1],y_test[2],y_test[3],y_test[4]], batch_size=128)
for num in range(0,len(model.metrics_names)):
    print(model.metrics_names[num]+': '+str(results[num]))

TypeError: Invalid keyword arguments: ['y_test']

In [46]:
predicted = model.predict(x_test)

261/261 [==============================] - 5s 13ms/step


In [68]:
from sklearn.metrics import confusion_matrix

In [73]:
y_test[0]

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]])

In [76]:
data_test

,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource
6185,1,1,0,0,0,"b'_compare_ints(_cs_io_t *inp1,\n ..."
5506,0,0,0,0,1,"b'fits_write_header(const qfits_header* hdr, c..."
1201,0,1,0,0,0,b'server_unlink (rpcsvc_request_t *req)\n{\n ...
3064,1,1,0,0,0,"b""hexdump(unsigned char *s, unsigned long l)\n..."
1077,0,0,0,0,1,b'GLESendSocket(const string& commands) {\n#if...
...,...,...,...,...,...,...
1727,0,0,0,1,0,b'qpol_constraint_get_perm_iter(const qpol_pol...
5756,0,1,0,0,1,"b'main(int argc, char ** argv)\n{\n dxml_elem..."
5697,0,1,0,0,0,"b""cob_move_alphanum_to_alphanum (cob_field *f1..."
8159,1,1,0,0,0,"b'git_path_find_dir(git_buf *dir, const char *..."


In [47]:
model.evaluate(x_test,y = [y_test[0],y_test[1],y_test[2],y_test[3],y_test[4]])

261/261 [==============================] - 4s 13ms/step - loss: 4.6806 - dense_2_loss: 0.9708 - dense_3_loss: 1.2906 - dense_4_loss: 0.3931 - dense_5_loss: 0.7554 - dense_6_loss: 1.2708 - dense_2_accuracy: 0.8765 - dense_3_accuracy: 0.8425 - dense_4_accuracy: 0.9637 - dense_5_accuracy: 0.9136 - dense_6_accuracy: 0.8128


[4.680647850036621,
 0.9707695245742798,
 1.2905772924423218,
 0.39306971430778503,
 0.7554128766059875,
 1.270817756652832,
 0.876471996307373,
 0.8424657583236694,
 0.9637106657028198,
 0.9136024713516235,
 0.8127853870391846]

In [52]:
model.save('vuldet.h5')

In [51]:
model = tf.keras.models.load_model("vuldet")

In [7]:

for col in range(1,6):
    print('\nThis is evaluation for column',col)
    confusion = sklearn.metrics.confusion_matrix(y_train=test.iloc[:,col].to_numpy(), y_pred=predicted[col-1])
    print(confusion)
    tn, fp, fn, tp = confusion.ravel()
    print('\nTP:',tp)
    print('FP:',fp)
    print('TN:',tn)
    print('FN:',fn)

    ## Performance measure
#     print('\nAccuracy: '+ str(sklearn.metrics.accuracy_score(y_true=test.iloc[:,col].to_numpy(), y_pred=pred_test[col-1])))
#     print('Precision: '+ str(sklearn.metrics.precision_score(y_true=test.iloc[:,col].to_numpy(), y_pred=pred_test[col-1])))
#     print('Recall: '+ str(sklearn.metrics.recall_score(y_true=test.iloc[:,col].to_numpy(), y_pred=pred_test[col-1])))
#     print('F-measure: '+ str(sklearn.metrics.f1_score(y_true=test.iloc[:,col].to_numpy(), y_pred=pred_test[col-1])))
#     print('Precision-Recall AUC: '+ str(sklearn.metrics.average_precision_score(y_true=test.iloc[:,col].to_numpy(), y_score=predicted[col-1][:,1])))
#     print('AUC: '+ str(sklearn.metrics.roc_auc_score(y_true=test.iloc[:,col].to_numpy(), y_score=predicted[col-1][:,1])))
#     print('MCC: '+ str(sklearn.metrics.matthews_corrcoef(y_true=test.iloc[:,col].to_numpy(), y_pred=pred_test[col-1])))


This is evaluation for column 1


NameError: name 'test' is not defined